# Flutter investigation

In [1]:
import numpy as np
import os
import pdb
import importlib
import cases.models_generator.gen_main as gm
import sharpy.utils.algebra as algebra
import sharpy.sharpy_main

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D

model_name = 'example_flutter'
model_route = os.getcwd()+'/'+model_name

print(model_route)

/home/pablodfs/FYP/Projects-SHARPy/aeroelasticPMOR_Optimization/flutter_investigation/example_flutter


## Model definition

In [2]:
# First some inputs
wing_semispan = 12 
bound_panels = 4 # This controls the chordwise discretisation of the aerogrid
components=['fuselage', 'wing_r', 'winglet_r',
                              'wing_l', 'winglet_l', 'vertical_tail',
                              'horizontal_tail_right', 'horizontal_tail_left']
# aeroelasticity parameters
main_ea = 0.23  # Wing elastic axis from LE as %
main_cg = 0.3  # Not sure about this input
sigma = 1.5
c_ref = 1.0

#########
# wings #
#########
#
ea = 1e7
ga = 1e5
gj = 1e4
eiy = 2e4
eiz = 4e6
m_bar_main = 0.75
j_bar_main = 0.075
mass_main1 = np.diag([m_bar_main, m_bar_main, m_bar_main,
                      j_bar_main, 0.5 * j_bar_main, 0.5 * j_bar_main])
stiffness_main1 = sigma * np.diag([ea, ga, ga, gj, eiy, eiz])
stiffness_main = np.zeros((1, 6, 6))
stiffness_main[0] = stiffness_main1
mass_main = np.zeros((1, 6, 6))
mass_main[0] = mass_main1
############
# fuselage #
############
#
sigma_fuselage = 10
m_bar_fuselage = 0.2
j_bar_fuselage = 0.08
stiffness_fuselage1 = np.diag([ea, ga, ga, gj, eiy, eiz]) * sigma * sigma_fuselage
stiffness_fuselage1[4, 4] = stiffness_fuselage1[5, 5]
mass_fuselage1 = np.diag([m_bar_fuselage,
                          m_bar_fuselage,
                          m_bar_fuselage,
                          j_bar_fuselage,
                          j_bar_fuselage * 0.5,
                          j_bar_fuselage * 0.5])
stiffness_fuselage = np.zeros((1, 6, 6))
stiffness_fuselage[0] = stiffness_fuselage1
mass_fuselage = np.zeros((1, 6, 6))
mass_fuselage[0] = mass_fuselage1
########
# tail #
########
#
sigma_tail = 100
m_bar_tail = 0.3
j_bar_tail = 0.08
stiffness_tail1 = np.diag([ea, ga, ga, gj, eiy, eiz]) * sigma * sigma_tail
stiffness_tail1[4, 4] = stiffness_tail1[5, 5]
mass_tail1 = np.diag([m_bar_tail,
                      m_bar_tail,
                      m_bar_tail,
                      j_bar_tail,
                      j_bar_tail * 0.5,
                      j_bar_tail * 0.5])
stiffness_tail = np.zeros((1, 6, 6))
stiffness_tail[0] = stiffness_tail1
mass_tail = np.zeros((1, 6, 6))
mass_tail[0] = mass_tail1

######################################
# Lumped mass at fuselage/wing cross #
######################################
n_lumped_mass = 1  # Number of lumped masses
lumped_mass_nodes = np.zeros((n_lumped_mass,), dtype=int)  # Maps lumped mass to nodes
lumped_mass = np.zeros((n_lumped_mass,))  # Array of lumped masses in kg
lumped_mass[0] = 50
lumped_mass_inertia = np.zeros((n_lumped_mass, 3, 3))  # 3x3 inertia to the previous masses
lumped_mass_position = np.zeros((n_lumped_mass, 3))  # Relative position to the belonging node in B FoR

##############
# Components #
##############
g1c = dict()
g1c['fuselage'] = {'workflow': ['create_structure', 'create_aero0'],
                   'geometry': {'length': 10,
                                'num_node': 9,
                                'direction': [1., 0., 0.],
                                'sweep': 0.,
                                'dihedral': 0.},
                   'fem': {'stiffness_db': stiffness_fuselage,
                           'mass_db': mass_fuselage,
                           'frame_of_reference_delta': [0, 1., 0.],
                           'lumped_mass': lumped_mass,
                           'lumped_mass_nodes': lumped_mass_nodes,
                           'lumped_mass_inertia': lumped_mass_inertia,
                           'lumped_mass_position': lumped_mass_position}
                   }

g1c['wing_r'] = {'workflow': ['create_structure', 'create_aero'],
                 'geometry': {'length': wing_semispan,
                              'num_node': 13,
                              'direction': [0., 1., 0.],
                              'sweep': 0. * np.pi / 180,
                              'dihedral': 0.},
                 'fem': {'stiffness_db': stiffness_main,
                         'mass_db': mass_main,
                         'frame_of_reference_delta': [-1, 0., 0.]},
                 'aero': {'chord': [1., 1.],
                          'elastic_axis': main_ea,
                          'surface_m': bound_panels}
                 }
g1c['winglet_r'] = {'workflow': ['create_structure', 'create_aero'],
                    'geometry': {'length': 4,
                                 'num_node': 5,
                                 'direction': [0., 1., 0.],
                                 'sweep': 0. * np.pi / 180,
                                 'dihedral': 20. * np.pi / 180},
                    'fem': {'stiffness_db': stiffness_main,
                            'mass_db': mass_main,
                            'frame_of_reference_delta': [-1, 0., 0.]},
                    'aero': {'chord': [1., 1.],
                             'elastic_axis': main_ea,
                             'surface_m': bound_panels,
                             'merge_surface': True}
                    }
g1c['wing_l'] = {'symmetric': {'component': 'wing_r'}}
g1c['winglet_l'] = {'symmetric': {'component': 'winglet_r'}}
g1c['vertical_tail'] = {'workflow': ['create_structure', 'create_aero'],
                        'geometry': {'length': 2.5,
                                     'num_node': 9,
                                     'direction': [0., 0., 1.],
                                     'sweep': None,
                                     'dihedral': None},
                        'fem': {'stiffness_db': stiffness_tail,
                                'mass_db': mass_tail,
                                'frame_of_reference_delta': [-1., 0., 0.]},
                        'aero': {'chord': [0.45, 0.45],
                                 'elastic_axis': 0.5,
                                 'surface_m': bound_panels}
                        }
g1c['horizontal_tail_right'] = {'workflow': ['create_structure', 'create_aero'],
                                'geometry': {'length': 2.5,
                                             'num_node': 9,
                                             'direction': [0., 1., 0.],
                                             'sweep': 0.,
                                             'dihedral': 0.},
                                'fem': {'stiffness_db': stiffness_tail,
                                        'mass_db': mass_tail,
                                        'frame_of_reference_delta': [-1, 0., 0.]},
                                'aero': {'chord': [0.5, 0.5],
                                         'elastic_axis': 0.5,
                                         'surface_m': bound_panels}
                                }
g1c['horizontal_tail_left'] = {'symmetric': {'component': 'horizontal_tail_right'}}

g1c_output = {i: g1c[i] for i in components}
    

## Model generation

In [3]:
# First some inputs
#model_name  defined in the first stage
g1mm = {'model_name': model_name,
        'model_route': model_route,
        # 'iterate_type': 'Full_Factorial',
        # 'write_iterate_vars': True,
        # 'iterate_vars': {'fuselage*geometry-length': np.linspace(7, 15., 3),
        #                  'wing_r*geometry-length': np.linspace(15, 25., 3),
        #                  'winglet_r*geometry-dihedral': np.pi / 180 * np.array([0, 20, 40])},
        # 'iterate_labels': {'label_type': 'number',
        #                    'print_name_var': 0},
        'assembly': {'include_aero': 1,
                     'default_settings': 1,  # beam_number and aero surface and
                     # surface_distribution
                     # selected by default one
                     # per component
                     'fuselage': {'upstream_component': '',
                                  'node_in_upstream': 0},
                     'wing_r': {'keep_aero_node': 1,
                                'upstream_component': 'fuselage',
                                'node_in_upstream': 0},
                     'winglet_r': {'keep_aero_node': 1,
                                   'upstream_component': 'wing_r',
                                   'node_in_upstream': -1},
                     'wing_l': {'upstream_component': 'fuselage',
                                'node_in_upstream': 0},
                     'winglet_l': {'upstream_component': 'wing_l',
                                   'node_in_upstream': -1},
                     'vertical_tail': {'upstream_component': 'fuselage',
                                       'node_in_upstream': -1},
                     'horizontal_tail_right': {'upstream_component': 'vertical_tail',
                                               'node_in_upstream': -1},
                     'horizontal_tail_left': {'upstream_component': 'vertical_tail',
                                              'node_in_upstream': -1}
                     }
        }
for ki in ['fuselage', 'wing_r', 'winglet_r',
           'wing_l', 'winglet_l', 'vertical_tail',
           'horizontal_tail_right', 'horizontal_tail_left']:

    if (ki not in ['include_aero', 'default_settings'] and
            ki not in components):
        del g1mm['assembly'][ki]

## Simulation definition

In [4]:
#############################################
#  Aeroelastic equilibrium                  #
#############################################
u_inf = 10
rho = 1.252
c_ref = 1.0
AoA_deg = 1.2
AoA = AoA_deg * np.pi / 180
bound_panels = 8

sol_112 = {
    'sharpy': {'simulation_input': None,
               'default_module': 'sharpy.routines.static',
               'default_solution': 'sol_112',
               'default_solution_vars': {
                   'u_inf': u_inf,
                   'rho': rho,
                   'gravity_on': False,
                   'dt': c_ref / bound_panels / u_inf,
                   'panels_wake': bound_panels * 5,
                   'rotationA': [0., AoA, 0.],
                   'horseshoe': True,
                   'fsi_maxiter': 100,
                   'fsi_tolerance': 1e-5,
                   'fsi_relaxation': 0.1,
                   'fsi_load_steps': 5,
                   's_maxiter': 100,
                   's_tolerance': 1e-5,
                   's_relaxation': 1e-3,
                   's_load_steps': 1,
                   's_delta_curved': 1e-4,
                   'add2_flow': [['StaticCoupled', ['plot', 'AeroForcesCalculator']]],
                   'AeroForcesCalculator': {'write_text_file': True},
                   # 'u_inf_direction': [np.cos(deg_to_rad(3.)),
                   #                     0., np.sin(deg_to_rad(3.))]
               },
               'default_sharpy': {},
               'model_route': None
               }
}

## Run Simulation

In [5]:
# For this single built model create the files required
folder2write = model_route
file2write = model_route+'/'+model_name

In [6]:
g1 = gm.Model('sharpy', ['sharpy'],
                      model_dict=g1mm,
                      components_dict=g1c_output,
                      #simulation_dict=define_sol_152(u_inf, AoA_deg[i], rho, bound_panels))
                      #simulation_dict=define_sol_0()
                      simulation_dict= sol_112)
# Create the file structure inside the folder
g1.build()  # Build the model
mi = 0
g1.built_models[mi].sharpy.sim = gm.Simulation(sim_type='sharpy',
                                               settings_sim=g1.simulation_dict['sharpy'],
                                               case_route=folder2write,
                                               case_name=g1.model_dict['model_name'])
g1.built_models[mi].sharpy.sim.get_sharpy(
    inp=g1.simulation_dict['sharpy']['simulation_input'])
g1.built_models[mi].sharpy.write_structure(file2write + '.fem.h5')
g1.built_models[mi].sharpy.write_aero(file2write + '.aero.h5')
g1.built_models[mi].sharpy.write_sim(file2write + '.sharpy')

os.chdir(model_route)
data = sharpy.sharpy_main.main(['', model_name + '.sharpy'])

No variable airfoil_efficiency defined in component fuselage
No variable airfoil_efficiency defined in component wing_r
No variable airfoil_efficiency defined in component winglet_r
No variable airfoil_efficiency defined in component wing_l
No variable airfoil_efficiency defined in component winglet_l
No variable airfoil_efficiency defined in component vertical_tail
No variable airfoil_efficiency defined in component horizontal_tail_right
No variable airfoil_efficiency defined in component horizontal_tail_left
--------------------------------------------------------------------------------
            ######  ##     ##    ###    ########  ########  ##    ##
           ##    ## ##     ##   ## ##   ##     ## ##     ##  ##  ##
           ##       ##     ##  ##   ##  ##     ## ##     ##   ####
            ######  ######### ##     ## ########  ########     ##
                 ## ##     ## ######### ##   ##   ##           ##
           ##    ## ##     ## ##     ## ##    ##  ##           ##
 

## From here try and build the flutter simulation to see what gets inputted

In [7]:
# Modal solution
from sharpy.solvers import modal as md

In [8]:
# Create an instance of the class
modal = md.Modal()

In [9]:
# Manually add the settings for the modal solution
print_info = False
keep_linear_matrices = True
continuous_eigenvalues = False
dt = 0.
delta_curved =1e-2
plot_eigenvalues = False
use_custom_timestep = -1
num_modes = 20
rigid_body_modes = False
rigid_modes_cg = False
use_undamped_modes = True
max_modal_disp = 0.15
max_modal_rot_deg = 15.
print_modal_matrices = True
write_modal_data = True
write_modes_vtk = True

# Note this can be done using the model generator
# From routines/modal use sol_132 for example which uses the 
# get_solver_sett func. from the routines/basic class
data.settings['Modal'] = {
    'NumLambda' : num_modes,
    'rigid_body_modes':rigid_body_modes,
    'rigid_modes_cg':rigid_modes_cg,
    'use_undamped_modes':use_undamped_modes,
    'max_displacement':max_modal_disp,
    'max_rotation_deg':max_modal_rot_deg,
    'print_matrices':print_modal_matrices,
    'write_dat':write_modal_data,
    'write_modes_vtk':write_modes_vtk,
    'print_info':print_info,
    'keep_linear_matrices':keep_linear_matrices,
    'continuous_eigenvalues':continuous_eigenvalues, 
    'dt':dt,
    'delta_curved':delta_curved, 
    'plot_eigenvalues':plot_eigenvalues, 
    'use_custom_timestep':use_custom_timestep 

}

In [10]:
print(type(data.settings['BeamLoader']))
print('\n')
print(data.settings['StaticCoupled'])
print('\n')
print(data.settings['Modal'])
print(type(data.settings['Modal']))

<class 'configobj.Section'>


{'n_load_steps': 5, 'max_iter': 100, 'tolerance': 1e-05, 'relaxation_factor': 0.1, 'aero_solver': 'StaticUvlm', 'structural_solver': 'NonLinearStatic', 'print_info': True, 'correct_forces_method': '', 'aero_solver_settings': {'print_info': True, 'horseshoe': True, 'num_cores': 1, 'n_rollup': 1, 'rollup_dt': 0.0125, 'rollup_aic_refresh': 1, 'rollup_tolerance': 0.0001, 'iterative_solver': False, 'iterative_tol': 0.0001, 'iterative_precond': False, 'velocity_field_generator': 'SteadyVelocityField', 'rho': 1.252, 'cfl1': True, 'vortex_radius': 1e-06, 'vortex_radius_wake_ind': 1e-06, 'rbm_vel_g': array([0., 0., 0., 0., 0., 0.]), 'centre_rot_g': array([0., 0., 0.]), 'velocity_field_input': {'u_inf': 10.0, 'u_inf_direction': array([1., 0., 0.])}}, 'structural_solver_settings': {'print_info': True, 'max_iterations': 100, 'num_load_steps': 1, 'delta_curved': 0.0001, 'min_delta': 1e-05, 'newmark_damp': 0.0001, 'gravity_on': False, 'gravity': 9.807, 'gravity_dir': ar

In [11]:
modal.initialise(data)

In [12]:
modal.run()

/home/pablodfs/sharpy/sharpy/utils/algebra.py:201: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[1, 2] = -vector[0]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:202: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[2, 0] = -vector[1]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:203: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[0, 1] = -vector[2]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:204: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[2, 1] = vector[0]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:205: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[0, 2] = vector[1]
/home/pablodfs/sharpy/sharpy/utils/algebra.py:206: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[1, 0] = vector[2]
/home/pablodfs/sharpy/sharpy/structure/utils/modalutils.py:178: ComplexWarning:

In [13]:
print(data.structure.timestep_info[0].modal)

{'modes': 'undamped', 'freq_natural': array([  2.740132755501345 +0.j,   2.7401327695799833+0.j,
        14.556078053072525 +0.j,  14.556078060400045 +0.j,
        16.702232854335286 +0.j,  16.70223287305978  +0.j,
        31.75222910446008  +0.j,  45.27916237444906  +0.j,
        45.27916237787907  +0.j,  45.81182002449558  +0.j,
        45.811820044524396 +0.j,  86.86316993627597  +0.j,
        86.8631699470782   +0.j, 119.79814941340005  +0.j,
       119.79814941918681  +0.j, 143.40451355033008  +0.j,
       143.4045135535516   +0.j, 145.00070129570042  +0.j,
       145.0007013086181   +0.j, 168.28154930557065  +0.j]), 'damping': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), 'eigenvalues': array([7.5083275177713933e+00+0.j, 7.5083275949260697e+00+0.j,
       2.1187940828713963e+02+0.j, 2.1187940850045953e+02+0.j,
       2.7896458232043699e+02+0.j, 2.7896458294591883e+02+0.j,
       1.0082040531021219e+03+0.j, 2.0502025453317233e+03+0

## Build the linear System

In [14]:
from sharpy.solvers import linearassembler as la
linear = la.LinearAssembler()


In [15]:
# Define settings for the linear assembler
linear_system = 'test_flutter' # Name of chosen state space assembly type
linear_system_settings = dict() # Settings for the desired state space assembler
linearisation_tstep = -1 # Chosen linearisation time step number from available time steps
modal_tstep = -1 # Timestep in which modal information is stored. Useful if the ``Modal`` solver' is run at the start of the SHARPy flow.
inout_coordinates = '' #Input/output coordinates of the system. Nodal or modal space.
# options = ['', 'nodes', 'modes']
retain_inputs = []
retain_outputs = []

data.settings['LinearAssembler'] = {
    'linear_system':linear_system, 
    'linear_system_settings':linear_system_settings, 
    'linearisation_tstep':linearisation_tstep, 
    'retain_inputs':retain_inputs, 
    'retain_outputs':retain_outputs,
    'modal_tstep':modal_tstep,
    'inout_coordinates':inout_coordinates
}

# Add settings_options to the instance of the class manually
linear.settings_options['inout_coordinates'] = ['','nodes','modes']

# custom_settings = {
#     'linear_system':linear_system, 
#     'linear_system_settings':linear_system_settings, 
#     'linearisation_tstep':linearisation_tstep, 
#     'retain_inputs':retain_inputs, 
#     'retain_outputs':retain_outputs 
# }

## Run linear initialise manually

In [16]:
data.settings['Modal']

{'NumLambda': 20, 'rigid_body_modes': False, 'rigid_modes_cg': False, 'use_undamped_modes': True, 'max_displacement': 0.15, 'max_rotation_deg': 15.0, 'print_matrices': True, 'write_dat': True, 'write_modes_vtk': True, 'print_info': False, 'keep_linear_matrices': True, 'continuous_eigenvalues': False, 'dt': 0.0, 'delta_curved': 0.01, 'plot_eigenvalues': False, 'use_custom_timestep': -1}

In [17]:
linear.data = data
from sharpy.utils.datastructures import Linear
import sharpy.utils.settings as settings     
import sharpy.linear.utils.ss_interface as ss_interface
linear.settings = data.settings[linear.solver_id]
settings.to_custom_types(linear.settings, linear.settings_types, linear.settings_default,
                         options=linear.settings_options, no_ctype=True)

# Get consistent linearisation timestep
ii_step = linear.settings['linearisation_tstep']

tsstruct0 = data.structure.timestep_info[ii_step]
tsaero0 = data.aero.timestep_info[ii_step]

try:
    tsstruct0.modal = data.structure.timestep_info[linear.settings['modal_tstep']].modal
except AttributeError:
    raise AttributeError('Unable to find modal information at desired '
                         'timestep {:g}'.format(linear.settings['modal_tstep']))

# Create data.linear
linear.data.linear = Linear(tsaero0, tsstruct0)

# Load available systems
import sharpy.linear.assembler

# Load roms
import sharpy.rom

lsys = ss_interface.initialise_system(linear.settings['linear_system'])
lsys.initialise(data)
linear.data.linear.linear_system = lsys

KeyError: 'test_flutter'

In [24]:
cls_type = dict_of_systems[linear.settings['linear_system']] = dict()
sys = cls_type()

TypeError: 'dict' object is not callable